# GradientClimber Demo

This notebook is adapted for use in the Gradient Climber application using the Garden Buddy application as an example.  We use the Gradient Climber Quality Attribute Scenarios as guidence for the required Properties and Conditions.

NOTE: this demo has an additional set of requirements than MLTE. You can install them with the command: 

`poetry install --with demo`


## 0. Quality Attribute Scenarios

The following are the QASs that we want to validate through the use of MLTE. The examples below relate to a reinforcement learning system based on the mountain car example [Andrew W. Moore (1990). Efficient Memory-Based Learning for Robot Control (PhD thesis, University of Cambridge)].

* **Interpretability - Model Produces Valid Outputs**
The model always produces outputs according to the output actions specified in the  documentation during normal operation


* **Functional Correctness - Model Does the Expected Thing**
Agent receives "start" command from system initialized with random position during Normal Operation. Agent moves car to desired location of x=0.6 within 250 time steps.
After initialization, the model successfully navigates the vehicle from a stationary position at the bottom of a hill, to the desired location (top of the hill in front of the vehicle) within the specified time bound.

* **Deployability - Model Can be Deployed into Productin When Needed**
New version of agent Q-table deployed onto device during Maintenance. New q-table is loaded into into available memory and system produces outputs within 16ms (60hz).

* **Reliability - Model Performs as Exected without Failure**
Model receives valid values from sensors during Normal Operation and produces outputs (actions) which improve the expected reward 99.9% of the time.
Model receives valid input values from from sensors during normal operation with the gradient climber system active. The model output is correct and improves the expected reward 99.9% of the time.

* **Portability - Model can be Adatped for Use on New Hardware/Software Platforms**
A new model is generated during the model training process. The model should be serialized in a format that can be moved across platforms with no need for retraining 

* **Compliance (Speed) - Model Adheres to Designated Ethical and Saftey Requirements**
The model receives a position and velocity input from *source* during normal operations. Regardless of input, the model will not produce output values that cause the vehicle to exceed the legal speed limit set in the operational configuration.

* **Compliance (Position) - Model Adheres to Designated Ethical and Saftey Requirements**
The model receives a position and velocity from *source* during normal operations. Regardless of input, the model will not produce output values that cause the vehicle to exit the area of operation set in the operational configuration.

# 1. Requirements

## 1.1 Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [1]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *

Creating initial custom lists at URI: local:///Users/jhansen/continuum/mlte/demo/GradientClimber/../store
Loaded 7 qa_categories for initial list
Loaded 30 quality_attributes for initial list
Creating sample catalog at URI: StoreType.LOCAL_FILESYSTEM:local:///Users/jhansen/continuum/mlte/demo/GradientClimber/../store
Loading sample catalog entries.
Loaded 17 entries for sample catalog.


## 1.2 Build a `NegotiationCard`

In MLTE, we negotiation requirements with the help of a `NegotiationCard`. This can be done manually through code, but it is easier to use the MLTE UI to do so. Below we are copying a pre-built one that applies to this scenario. In MLTE, we define requirements by constructing a `NegotiationCard` that will include explicit Quality Attribute Scenarios with the requirements.

In [2]:
%%bash
(cd ../ && cp -r ./sample_store/* ./store/)

## 1.3 Define a TestSuite

In the first phase of SDMT, we define a `TestSuite` that represents the tests the completed model must will have to pass in order to be acceptable for use in the system into which it will be integrated.

In MLTE, we define the tests that will be required for the different requirements in a `TestSuite`. Note that a new `Evidence` types (`MultipleRanksums`) had to be created in this case to handle the data and `Validator` for that case, and two stand-alone `Validator`s were defined in `validators.py` to validate data using existing `Evidence` types.

First we need to load up our `NegotiationCard`, so we can get the list of ids of its quaity attribute scenarios, that will be added to the `TestCase`s here. Those ids are the way to link the `TestCase`s to their quality attribute requirements.

In [3]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
card.save(force=True)
card.print_quality_scenarios()

default.card-qas_001 (Functional Correctness): The gradient climber system gets an initialization command from the user while in normal operation, the model successfully navigates the vehicle, to the goal position within the desired time bound
default.card-qas_002 (Reliability): Model receives valid input values from sensors while in normal operations., the model output is correct, and improves the expected reward 99.9% of the time
default.card-qas_003 (Compliance, position): Regardless of input values from sensors while in normal operation, the model will produce output values that ensure the vehicle, remains in the area of operation specified in the set operational configuration.
default.card-qas_004 (Compliance, velocity): Regardless of input values  from sensors while in normal operation, the model will produce output values that ensure the vehicle, speed does not exceed limit.
default.card-qas_006 (Deployability): When a new version of a model is made available following retrainin

Now we can create our `TestSuite`, consisting of a list of `TestCases`, each of them addressing one or more Quality Attribute Scenarios from our `NegotiationCard`. When defining the `TestCase`s below, we need to set the id of the corresponding Quality Attribute Scenario we want to test in its "quality_scenarios" attribute.

In [ ]:
from mlte.tests.test_case import TestCase
from mlte.tests.test_suite import TestSuite

# The Evidence types we will use to validate each condition.
from utils import validators
from mlte.validation.validator import Validator

test_suite = TestSuite(
    test_cases=[
        # Functional correctness
        TestCase(
            identifier="functional correctness",
            goal="Ensure that the model navigates vehicle to goal location within required time",
            quality_scenarios=["default.card-qas_001"],
            validator=validators.passed_percent_more_or_equal_then(0.9),
        ),
        # Reliability
        TestCase(
            identifier="reliability",
            goal="Ensure model outputs an action that improves expected reward 99.9% of the time",
            quality_scenarios=["default.card-qas_002"],
            validator=validators.passed_percent_more_or_equal_then(0.999),
        ),
        # Compliance (position)
        TestCase(
            identifier="gradient climber position accuracy",
            goal="Check if model complies with position requirements",
            quality_scenarios=["default.card-qas_003"],
            validator=validators.passed_percent_more_or_equal_then(1.0),
        ),
        # Compliance (velocity)
        TestCase(
            identifier="gradient climber velocity accuracy",
            goal="Check if model complies with position requirements",
            quality_scenarios=["default.card-qas_004"],
            validator=validators.passed_percent_more_or_equal_then(1.0),
        ),
        # Deployability
        TestCase(
            identifier="deployability",
            goal="Verified that the q-table file is available and of a manageable size.",
            quality_scenarios=["default.card-qas_006"],
            validator=Validator.build_info_validator(
                "Inspect project code and documentation."
            ),
        ),
        # Portability
        TestCase(
            identifier="portability",
            goal="Data is in the form of an npy file which is portable by design.",
            quality_scenarios=["default.card-qas_007"],
            validator=Validator.build_info_validator(
                "Inspect project code and documentation."
            ),
        ),
    ]
)
# The full test suite.

test_suite.save(parents=True, force=True)

RuntimeError: Quality Attribute Scenario with id default.card-qas_005 was not found on any NegotiationCard